In [1]:
import pymongo
from pymongo import MongoClient

In [2]:
#step=1 connect to mongodb database
client=MongoClient()
# or client=MongoClient('localhost',27017)

for d in client.list_database_names():
    print(d)

admin
config
db5556
local


In [3]:
#connecting to one particular database
#db=client.db5556
db=client['admin']
print(db.list_collection_names())

['system.version']


In [6]:
#retriving records from database
cursor=db.system.version.find()
for doc in cursor:
    print(doc['version'])      
    #print(doc)


6.0


In [ ]:
import json
import base64
import streamlit as st
import pandas as pd
from pymongo import MongoClient
from tscrape import scrape_twitter_data


# Define a function to upload data to MongoDB
def upload_to_mongodb(json_file):
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017')
    db = client['twitter_scraping']
    collection = db['scrapped1']
    # Convert data to JSON records and Insert data into MongoDB
    with open(json_file, 'r') as f:
        data = json.load(f)
    result = collection.insert_many(data)

    # Retrieve the uploaded data from MongoDB
    uploaded_data = list(collection.find())

    return uploaded_data


# Define the Streamlit app
def main():
    # Page title
    st.title('Twitter Scraper')

    # Get user input
    hashtag = st.text_input('Hashtag/Keyword')
    start_date = st.date_input('Start Date')
    end_date = st.date_input('End Date')
    max_tweets = st.number_input('Max Tweets')

    # Scrape Twitter data
    if st.button('Scrape'):
        tweets_df = scrape_twitter_data(hashtag, start_date, end_date, max_tweets)

        # Create JSON file
        json_file = 'tweets.json'
        tweets_df.to_json(json_file, orient='records')

        # Display data in a table
        st.write(tweets_df)

        # Download data in CSV format
        csv = tweets_df.to_csv(index=False)
        b64 = base64.b64encode(csv.encode()).decode()
        st.markdown('### Download CSV File')
        href = f'<a href="data:file/csv;base64,{b64}" download="twitter_data.csv">Download CSV</a>'
        st.markdown(href, unsafe_allow_html=True)

        # Download data in JSON format
        json = tweets_df.to_json(orient='records')
        b64 = base64.b64encode(json.encode()).decode()
        st.markdown('### Download JSON File')
        href = f'<a href="data:file/json;base64,{b64}" download="twitter_data.json">Download JSON</a>'
        st.markdown(href, unsafe_allow_html=True)

        # Upload data to MongoDB
        if st.button('Upload to MongoDB'):
            uploaded_data = upload_to_mongodb(json_file)
            # Display result
            st.write(f'{len(uploaded_data)} documents uploaded to MongoDB')
            # Display the uploaded data in a table
            st.write(pd.DataFrame(uploaded_data))


if __name__ == '__main__':
    main()
